In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

In [ ]:
!nvidia-smi

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


## LLaMA2 7B Chat


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

In [ ]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
!nvidia-smi

In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest  legal assistant based in India with expertise in Indian law. You have thorough understanding of IPC
 ( Indian penal code), Crpc (Code of Criminal Procedure) the constitution of india and all the legal related stuff national and state wise of india.
Always answer as helpfully as possible, while being safe.
Please ensure that your responses are socially unbiased and factual in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering
something not correct. If you don't know the answer to a question, please don't share false information.

"""




def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
instruction = """ You will extract all the relevant information and save it in a database. This is a sample fir template to consider:

Police Station:
District:

1. Personal details of the Complainant / Informant:
(a) Name:
(b) Father's / Husband's Name: {
(c) Address:
(d) Phone number & Fax:
(e) Email:

2. Place of Occurrence:

3. Date and Hour of Occurrence:

4. Offence:
(a) Nature of the offence:
(b) Section:
(c) Particulars of the property:

5. Description of the accused:

6. Additional Section Information:
   - Bailable:
   - Cognizable:
   - Court:
   - Punishment:

7. Details of witnesses (if any):

8. Complaint:

You have to extract all of the information corresponding to the given keywords above.
"""

get_prompt(instruction)

In [ ]:
instruction = "Print the extracted inforamtion in a clear, proffesional and human readable form, matching the FIR format."

system_prompt = "You are a law expert"

get_prompt(instruction, system_prompt)

## LangChain basics

### Summarization

In [ ]:

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
output = llm_chain.run(text)
print(count_words(output))
parse_text(output)

## Simple Chatbot

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [ ]:
instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input}"
system_prompt = "You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

In [ ]:

prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
llm_chain.predict(user_input="Hi, my name is Sam")

In [ ]:
llm_chain.predict(user_input="Can you tell me about yourself.")

In [ ]:
llm_chain.predict(user_input="Today is Friday. What number day of the week is that?")

In [ ]:
llm_chain.predict(user_input="what is the day today?")

In [ ]:
llm_chain.predict(user_input="What is my name?")

In [ ]:
llm_chain.predict(user_input="Can you tell me about the olympics")

In [ ]:
llm_chain.predict(user_input="What have we talked about in this Chat?")